In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why Unet
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_why_unet.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# U-Net and Full DDPM Training — Vizuara

## 1. Why Does This Matter?

In the previous notebooks, we built the forward diffusion process and trained a simple CNN to predict noise. But simple CNNs are not powerful enough to generate realistic images. The DDPM paper uses a **U-Net architecture** — an encoder-decoder network with skip connections that can process images at multiple resolutions while preserving fine spatial details.

In this notebook, we will:
- Build a complete U-Net noise predictor with timestep conditioning
- Understand why skip connections and multi-scale processing are essential
- Train a full DDPM on MNIST and generate images from pure noise
- Implement the complete sampling loop

This is where everything comes together — by the end, you will have a working diffusion model that generates handwritten digits from scratch.

## 2. Building Intuition

Why do we need a U-Net specifically? Consider what the noise predictor must do:

At **high noise levels** (large $t$), the image is almost pure noise. The model needs to make **global decisions** — is this going to be a "3" or a "7"? This requires understanding the big picture.

At **low noise levels** (small $t$), the image is mostly clean. The model needs to refine **local details** — sharpening edges, fixing individual pixels. This requires fine-grained spatial information.

The U-Net handles both by processing at multiple resolutions:
- **Encoder (downsampling):** Captures global structure by progressively reducing resolution
- **Decoder (upsampling):** Reconstructs details by progressively increasing resolution
- **Skip connections:** Pass fine-grained spatial information directly from encoder to decoder

Let us build it step by step.

In [ ]:
#@title 🎧 Code Walkthrough: Install And Params
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_install_and_params.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
!pip install torch torchvision matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Setup diffusion parameters
T = 1000
betas = torch.linspace(1e-4, 0.02, T).to(device)
alphas = (1.0 - betas).to(device)
alpha_bars = torch.cumprod(alphas, dim=0).to(device)

def forward_diffusion(x_0, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_0)
    sqrt_ab = torch.sqrt(alpha_bars[t]).view(-1, 1, 1, 1)
    sqrt_1_ab = torch.sqrt(1 - alpha_bars[t]).view(-1, 1, 1, 1)
    x_t = sqrt_ab * x_0 + sqrt_1_ab * noise
    return x_t, noise

# Load MNIST (padded to 32x32 for cleaner U-Net downsampling)
transform = transforms.Compose([
    transforms.Pad(2),  # 28x28 -> 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
#@title 🎧 Listen: Timestep Embedding Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_timestep_embedding_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Sinusoidal Timestep Embedding

The model needs to know the noise level. We embed the timestep $t$ using sinusoidal positional encodings (the same idea from Transformers):

$$\text{PE}(t, 2i) = \sin(t / 10000^{2i/d})$$
$$\text{PE}(t, 2i+1) = \cos(t / 10000^{2i/d})$$

Let us plug in numbers. For $t=500$, $d=64$, $i=0$:
- $\text{PE}(500, 0) = \sin(500 / 10000^{0/64}) = \sin(500/1) = \sin(500) \approx -0.468$
- $\text{PE}(500, 1) = \cos(500 / 10000^{0/64}) = \cos(500) \approx 0.884$

For $i=16$: $10000^{32/64} = 10000^{0.5} = 100$
- $\text{PE}(500, 32) = \sin(500/100) = \sin(5) \approx -0.959$

Different $i$ values create features that oscillate at different frequencies, giving the model a rich representation of the timestep.

### U-Net Architecture Summary

The U-Net processes at 4 resolution levels: 32x32, 16x16, 8x8, and 4x4. At each level:
1. Two convolutional blocks with time embedding injection
2. Downsampling (encoder) or upsampling (decoder)
3. Skip connections between matching encoder/decoder levels

## 4. Let's Build It — Component by Component

### Step 1: Sinusoidal Timestep Embedding

In [ ]:
#@title 🎧 Code Walkthrough: Sinusoidal Embedding Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_sinusoidal_embedding_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SinusoidalTimeEmbedding(nn.Module):
    """Sinusoidal positional embedding for timesteps."""

    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None].float() * emb[None, :]
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return emb

In [ ]:
#@title 🎧 What to Look For: Visualize Embedding
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_visualize_embedding.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the time embedding
time_emb = SinusoidalTimeEmbedding(64)
t_values = torch.arange(0, T, 10)
embeddings = time_emb(t_values).detach().numpy()

plt.figure(figsize=(12, 4))
plt.imshow(embeddings.T, aspect='auto', cmap='RdBu')
plt.xlabel('Timestep t')
plt.ylabel('Embedding Dimension')
plt.title('Sinusoidal Timestep Embeddings')
plt.colorbar(label='Value')
plt.show()
print("Each column is a unique embedding — the model can distinguish all 1000 timesteps.")

In [ ]:
#@title 🎧 Code Walkthrough: Residual Block
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_residual_block.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Step 2: Residual Convolutional Block

In [ ]:
class ResBlock(nn.Module):
    """Residual block with time embedding injection."""

    def __init__(self, in_ch, out_ch, time_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.time_mlp = nn.Linear(time_dim, out_ch)
        self.norm1 = nn.GroupNorm(8, out_ch)
        self.norm2 = nn.GroupNorm(8, out_ch)

        # Shortcut if dimensions change
        self.shortcut = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x, t_emb):
        h = self.norm1(F.relu(self.conv1(x)))

        # Inject time embedding
        t = F.relu(self.time_mlp(t_emb))
        h = h + t[:, :, None, None]

        h = self.norm2(F.relu(self.conv2(h)))
        return h + self.shortcut(x)

In [ ]:
#@title 🎧 Code Walkthrough: Full Unet
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_full_unet.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Step 3: Full U-Net

In [ ]:
class UNet(nn.Module):
    """
    U-Net for DDPM noise prediction.

    Architecture:
    - 4 resolution levels: 32 -> 16 -> 8 -> 4
    - Channel progression: 1 -> 64 -> 128 -> 256 -> 512
    - Skip connections between matching encoder/decoder levels
    """

    def __init__(self, in_ch=1, base_ch=64, time_dim=256):
        super().__init__()

        # Time embedding
        self.time_embed = nn.Sequential(
            SinusoidalTimeEmbedding(base_ch),
            nn.Linear(base_ch, time_dim),
            nn.ReLU(),
            nn.Linear(time_dim, time_dim),
        )

        # Encoder (downsampling path)
        self.enc1 = ResBlock(in_ch, base_ch, time_dim)          # 32x32
        self.enc2 = ResBlock(base_ch, base_ch * 2, time_dim)    # 16x16
        self.enc3 = ResBlock(base_ch * 2, base_ch * 4, time_dim)  # 8x8

        self.down1 = nn.Conv2d(base_ch, base_ch, 4, stride=2, padding=1)
        self.down2 = nn.Conv2d(base_ch * 2, base_ch * 2, 4, stride=2, padding=1)
        self.down3 = nn.Conv2d(base_ch * 4, base_ch * 4, 4, stride=2, padding=1)

        # Bottleneck (4x4)
        self.bottleneck = ResBlock(base_ch * 4, base_ch * 4, time_dim)

        # Decoder (upsampling path)
        self.up3 = nn.ConvTranspose2d(base_ch * 4, base_ch * 4, 4, stride=2, padding=1)
        self.up2 = nn.ConvTranspose2d(base_ch * 2, base_ch * 2, 4, stride=2, padding=1)
        self.up1 = nn.ConvTranspose2d(base_ch, base_ch, 4, stride=2, padding=1)

        # Decoder blocks (input channels doubled due to skip connections)
        self.dec3 = ResBlock(base_ch * 8, base_ch * 2, time_dim)   # 8x8
        self.dec2 = ResBlock(base_ch * 4, base_ch, time_dim)       # 16x16
        self.dec1 = ResBlock(base_ch * 2, base_ch, time_dim)       # 32x32

        # Final output
        self.final = nn.Conv2d(base_ch, in_ch, 1)

    def forward(self, x, t):
        # Time embedding
        t_emb = self.time_embed(t)

        # Encoder
        e1 = self.enc1(x, t_emb)        # 32x32, base_ch
        e2 = self.enc2(self.down1(e1), t_emb)  # 16x16, base_ch*2
        e3 = self.enc3(self.down2(e2), t_emb)  # 8x8, base_ch*4

        # Bottleneck
        b = self.bottleneck(self.down3(e3), t_emb)  # 4x4, base_ch*4

        # Decoder with skip connections
        d3 = self.dec3(torch.cat([self.up3(b), e3], dim=1), t_emb)   # 8x8
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1), t_emb)  # 16x16
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1), t_emb)  # 32x32

        return self.final(d1)


model = UNet(in_ch=1, base_ch=64).to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"U-Net parameters: {total_params:,}")

In [ ]:
#@title 🎧 Narration: Unet Test
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_unet_test.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Quick test: verify the model produces correct output shape
test_x = torch.randn(4, 1, 32, 32).to(device)
test_t = torch.randint(0, T, (4,)).to(device)
test_out = model(test_x, test_t)
print(f"Input shape:  {test_x.shape}")
print(f"Output shape: {test_out.shape}")
assert test_x.shape == test_out.shape, "Input and output shapes must match!"
print("Shape check passed!")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** The model should have roughly 5-10 million parameters and produce output with the same shape as the input.

## 5. Your Turn

### TODO 1: Add Self-Attention to the Bottleneck

Self-attention allows the model to capture long-range dependencies. This is especially useful at low resolutions where global context matters most.

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SelfAttention(nn.Module):
    """
    TODO: Implement self-attention for the U-Net bottleneck.

    1. Project input to Q, K, V using 1x1 convolutions
    2. Reshape to (B, C, H*W) for attention computation
    3. Compute attention: softmax(Q^T K / sqrt(d)) * V
    4. Reshape back and add residual connection

    Args:
        channels: Number of input/output channels
    """
    def __init__(self, channels):
        super().__init__()
        # HINT: Use nn.Conv2d with kernel_size=1 for Q, K, V projections
        # HINT: Scale attention by 1/sqrt(channels)

        pass  # YOUR CODE HERE

    def forward(self, x):
        # HINT: Remember to reshape from (B,C,H,W) to (B,C,H*W) for attention
        pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement Gradient Accumulation for Larger Effective Batch Size

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def train_with_gradient_accumulation(model, dataloader, optimizer, accum_steps=4, num_epochs=1):
    """
    TODO: Train with gradient accumulation to simulate larger batch sizes.

    1. Zero gradients at the start
    2. For each batch, compute loss and call loss.backward()
    3. Only call optimizer.step() every accum_steps batches
    4. Scale the loss by 1/accum_steps before backward

    This lets you train with effective batch size = actual_batch * accum_steps
    without needing more GPU memory.
    """
    # HINT: Use (batch_idx + 1) % accum_steps == 0 to decide when to step

    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Code Walkthrough: Training Loop
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_training_loop.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
# Complete training loop
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
losses = []

print("Training DDPM...")
for epoch in range(10):
    model.train()
    epoch_losses = []

    for batch_idx, (images, _) in enumerate(dataloader):
        images = images.to(device)
        batch_size = images.shape[0]

        # Sample random timesteps
        t = torch.randint(0, T, (batch_size,), device=device)

        # Sample noise and create noisy images
        noise = torch.randn_like(images)
        x_t, _ = forward_diffusion(images, t, noise)

        # Predict noise
        predicted_noise = model(x_t, t)

        # DDPM loss
        loss = F.mse_loss(predicted_noise, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
        if batch_idx % 200 == 0:
            print(f"  Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")

    avg_loss = np.mean(epoch_losses)
    losses.extend(epoch_losses)
    print(f"Epoch {epoch+1}/10 — Average Loss: {avg_loss:.4f}")

In [ ]:
#@title 🎧 What to Look For: Plot Loss
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_plot_loss.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot training loss
plt.figure(figsize=(12, 4))
plt.plot(np.convolve(losses, np.ones(50)/50, mode='valid'))
plt.xlabel('Training Steps')
plt.ylabel('MSE Loss (smoothed)')
plt.title('DDPM Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
#@title 🎧 Transition: Sampling Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_sampling_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

### Sampling: Generating Images from Noise

In [ ]:
#@title 🎧 Code Walkthrough: Sample Ddpm Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_sample_ddpm_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
@torch.no_grad()
def sample_ddpm(model, n_samples=16, img_shape=(1, 32, 32)):
    """Generate images using the trained DDPM."""
    model.eval()

    # Start from pure noise
    x = torch.randn(n_samples, *img_shape, device=device)

    for t in reversed(range(T)):
        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)

        # Predict noise
        predicted_noise = model(x, t_batch)

        # Compute denoised image
        alpha_t = alphas[t]
        alpha_bar_t = alpha_bars[t]
        beta_t = betas[t]

        x = (1 / torch.sqrt(alpha_t)) * (
            x - (beta_t / torch.sqrt(1 - alpha_bar_t)) * predicted_noise
        )

        # Add noise (except at last step)
        if t > 0:
            x = x + torch.sqrt(beta_t) * torch.randn_like(x)

    return x

In [ ]:
#@title 🎧 What to Look For: Generate Samples
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_generate_samples.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Generate samples
print("Generating samples (this takes a minute)...")
samples = sample_ddpm(model, n_samples=16)

fig, axes = plt.subplots(2, 8, figsize=(24, 6))
for i, ax in enumerate(axes.flat):
    img = samples[i].squeeze().cpu().clamp(-1, 1).numpy()
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.suptitle('Generated Samples from Trained DDPM', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Visualize Trajectory
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_visualize_trajectory.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the progressive denoising for a single sample
@torch.no_grad()
def sample_with_trajectory(model, img_shape=(1, 32, 32), save_every=100):
    """Generate one image and save intermediate steps."""
    model.eval()
    x = torch.randn(1, *img_shape, device=device)
    trajectory = [x.clone()]

    for t in reversed(range(T)):
        t_batch = torch.tensor([t], device=device)
        predicted_noise = model(x, t_batch)

        alpha_t = alphas[t]
        alpha_bar_t = alpha_bars[t]
        beta_t = betas[t]

        x = (1 / torch.sqrt(alpha_t)) * (
            x - (beta_t / torch.sqrt(1 - alpha_bar_t)) * predicted_noise
        )
        if t > 0:
            x = x + torch.sqrt(beta_t) * torch.randn_like(x)

        if t % save_every == 0:
            trajectory.append(x.clone())

    return trajectory

trajectory = sample_with_trajectory(model, save_every=100)

fig, axes = plt.subplots(1, len(trajectory), figsize=(3 * len(trajectory), 3))
for i, (ax, img) in enumerate(zip(axes, trajectory)):
    t_val = T - i * 100 if i < len(trajectory) - 1 else 0
    ax.imshow(img.squeeze().cpu().clamp(-1, 1).numpy(), cmap='gray')
    ax.set_title(f't={t_val}')
    ax.axis('off')
plt.suptitle('Progressive Denoising — From Pure Noise to Generated Image', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Final Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_final_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
# Generate a beautiful 4x8 grid of diverse samples
print("Generating final sample grid...")
final_samples = sample_ddpm(model, n_samples=32)

fig, axes = plt.subplots(4, 8, figsize=(24, 12))
for i, ax in enumerate(axes.flat):
    img = final_samples[i].squeeze().cpu().clamp(-1, 1).numpy()
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.suptitle('DDPM Generated Digits — Trained from Scratch on MNIST', fontsize=18)
plt.tight_layout()
plt.savefig('ddpm_generated_samples.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nModel: U-Net with {total_params:,} parameters")
print(f"Training: {len(losses)} steps, final loss: {losses[-1]:.4f}")
print(f"Sampling: {T} denoising steps per image")
print("Generated 32 unique digits from pure Gaussian noise!")

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Key Takeaways
1. The U-Net's multi-scale architecture with skip connections is ideal for noise prediction
2. Sinusoidal time embeddings let the model adapt its behavior to different noise levels
3. Training is just MSE between true and predicted noise — no adversarial training needed
4. Sampling requires T = 1000 forward passes, which is slow but produces high-quality results

### Reflection Questions
- Why do skip connections improve generation quality? What would happen without them?
- How does the model "decide" what digit to generate when starting from random noise?
- What are the trade-offs between using more diffusion steps T and fewer?
- How would you modify this architecture for color images (3 channels)?

### What is Next
In the final notebook, we will explore the connection between DDPM and score-based generative models, implement faster sampling with DDIM, and discuss how this framework scales to produce stunning results like Stable Diffusion and DALL-E 2.